# Import librerie e training set

In [140]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [141]:
df = pd.read_csv('ML-CUP22-TR.csv', header=None, skiprows=7, index_col=0)
df.head()

#fare holdout

,1,2,3,4,5,6,7,8,9,10,11
0,,,,,,,,,,,
1,-1.704815,1.503106,-0.817083,1.353560,-1.293660,1.362608,0.217795,0.462728,-1.237246,22.222923,-27.036774
2,-0.888962,0.711473,-0.480023,0.748175,-0.981763,1.765482,-0.445018,1.719143,-1.096250,18.885520,-28.377440
3,1.143802,-1.591273,0.670477,-0.948551,1.075740,-0.908309,-0.694223,-1.321391,0.662656,4.587627,-18.507566
4,0.630938,-0.295533,1.379162,-0.041045,-0.250695,0.025003,0.143915,0.861455,0.879540,10.256820,-30.017064
5,1.799449,-0.913254,1.245698,-1.410940,0.653328,-0.677108,-0.667490,-0.849795,0.497977,4.916182,-17.670030


In [142]:
df.shape

(1492, 11)

In [143]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1492 entries, 1 to 1492
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   1       1492 non-null   float64
 1   2       1492 non-null   float64
 2   3       1492 non-null   float64
 3   4       1492 non-null   float64
 4   5       1492 non-null   float64
 5   6       1492 non-null   float64
 6   7       1492 non-null   float64
 7   8       1492 non-null   float64
 8   9       1492 non-null   float64
 9   10      1492 non-null   float64
 10  11      1492 non-null   float64
dtypes: float64(11)
memory usage: 139.9 KB


In [144]:
df.dtypes

1     float64
2     float64
3     float64
4     float64
5     float64
6     float64
7     float64
8     float64
9     float64
10    float64
11    float64
dtype: object

In [145]:
X = df.iloc[:, 0:9].values
X

array([[-1.704815,  1.503106, -0.817083, ...,  0.217795,  0.462728,
        -1.237246],
       [-0.888962,  0.711473, -0.480023, ..., -0.445018,  1.719143,
        -1.09625 ],
       [ 1.143802, -1.591273,  0.670477, ..., -0.694223, -1.321391,
         0.662656],
       ...,
       [ 0.22256 ,  0.171394,  2.338296, ...,  0.61909 ,  0.448008,
         0.134578],
       [ 1.484195, -1.6164  , -0.30885 , ..., -0.213782, -1.155141,
         0.681062],
       [-1.701126,  0.88336 , -0.332723, ...,  0.032551,  0.518337,
        -1.39578 ]])

In [146]:
y = df.iloc[:, 9:].values
y

array([[ 22.222923, -27.036774],
       [ 18.88552 , -28.37744 ],
       [  4.587627, -18.507566],
       ...,
       [ 10.887319, -30.49842 ],
       [  4.286232, -23.027878],
       [ 18.453878, -28.434854]])

# Partitioning

In [147]:
from sklearn.model_selection import train_test_split

In [148]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=0)

In [149]:
y_train

array([[  9.768276, -29.391799],
       [  8.237292, -28.261941],
       [ 18.811881, -28.325599],
       ...,
       [ 15.392804, -29.572999],
       [ 20.092161, -27.876527],
       [  5.06925 , -19.305946]])

In [150]:
y1_train = np.array([i[0] for i in y_train])

In [151]:
y2_train = np.array([i[1] for i in y_train])

In [152]:
y1_test = np.array([i[0] for i in y_test])
y2_test = np.array([i[1] for i in y_test])

# Model selection

In [186]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV, RandomizedSearchCV, KFold
from sklearn.svm import SVR
from scipy.stats import uniform
from sklearn.metrics import mean_squared_error, make_scorer, mean_absolute_error, r2_score
import sklearn.metrics

In [154]:
[2**i for i in range(-4, 1)]

[0.0625, 0.125, 0.25, 0.5, 1]

In [163]:
param_grid = [{
    'kernel': ['linear', 'rbf', 'sigmoid'],
    'C' : [2**i for i in range(-3, 1)], # Regularization/missclassification term: tells how much error is bearable
    'gamma' : [2**i for i in range(-3, 1)],
    'epsilon' : [0.0001, 0.001, 0.01, 0.1, 1] # specifies the epsilon-tube within which no penalty is associated in the training loss function with points predicted within a distance epsilon from the actual value
    #'coef0' : [0.0001, 0.001, 0.01, 0.1, 1, 2], # only significant in poly and sigmoid
    #'tol' : [0.0001, 0.001, 0.01, 1, 2],
},
{
    'kernel' : ['poly'],
    'C' : [2**i for i in range(-3, 1)],
    'degree': [i for i in range(4)],
    'gamma' : [2**i for i in range(-3, 1)],
    'epsilon' : [0.0001, 0.001, 0.01, 0.1, 1]
    #'coef0' : [0.0001, 0.001, 0.01, 0.1, 1, 2],
    #'tol' : [0.0001, 0.001, 0.01, 1, 2],
    #'epsilon' : [0.0001, 0.001, 0.01, 0.1, 1, 2]
}]

In [164]:
def mean_euclidean_error(T, O):
    sum = 0
    for t, o in zip(T, O):
        sum += np.linalg.norm(t - o)
    return sum / T.shape[0]

In [165]:
grid = GridSearchCV( 
    SVR(),
    param_grid=param_grid,
    cv=KFold(n_splits=5, shuffle=True, random_state=42),
    scoring = make_scorer(mean_euclidean_error, greater_is_better=False)
    )

In [169]:
grid.fit(X_train, y1_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=SVR(),
             param_grid=[{'C': [0.125, 0.25, 0.5, 1],
                          'epsilon': [0.0001, 0.001, 0.01, 0.1, 1],
                          'gamma': [0.125, 0.25, 0.5, 1],
                          'kernel': ['linear', 'rbf', 'sigmoid']},
                         {'C': [0.125, 0.25, 0.5, 1], 'degree': [0, 1, 2, 3],
                          'epsilon': [0.0001, 0.001, 0.01, 0.1, 1],
                          'gamma': [0.125, 0.25, 0.5, 1], 'kernel': ['poly']}],
             scoring=make_scorer(mean_euclidean_error, greater_is_better=False))

In [170]:
grid.best_params_

{'C': 1, 'epsilon': 0.1, 'gamma': 0.125, 'kernel': 'rbf'}

In [171]:
grid.best_score_

-0.8469289685110457

In [180]:
best_y1_model = SVR(kernel='rbf', C=1, epsilon=0.1, gamma=0.125)

In [192]:
pred0 = best_y1_model.predict(X_test)

In [193]:
mean_euclidean_error(pred0, y1_test)

0.8044007241768238

In [194]:
r2_score(pred0, y1_test)

0.9750114252495621

In [195]:
mean_squared_error(pred0, y1_test)

1.0458159628453512

In [196]:
mean_absolute_error(pred0, y1_test)

0.8044007241768237

In [174]:
grid.fit(X_train, y2_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=SVR(),
             param_grid=[{'C': [0.125, 0.25, 0.5, 1],
                          'epsilon': [0.0001, 0.001, 0.01, 0.1, 1],
                          'gamma': [0.125, 0.25, 0.5, 1],
                          'kernel': ['linear', 'rbf', 'sigmoid']},
                         {'C': [0.125, 0.25, 0.5, 1], 'degree': [0, 1, 2, 3],
                          'epsilon': [0.0001, 0.001, 0.01, 0.1, 1],
                          'gamma': [0.125, 0.25, 0.5, 1], 'kernel': ['poly']}],
             scoring=make_scorer(mean_euclidean_error, greater_is_better=False))

In [175]:
grid.best_params_

{'C': 1, 'epsilon': 0.1, 'gamma': 0.25, 'kernel': 'rbf'}

In [176]:
grid.best_score_

-1.038926582926158

In [197]:
pred1 = grid.best_estimator_.predict(X_test)

In [198]:
mean_euclidean_error(pred1, y2_test)

0.94876182439009

In [199]:
r2_score(pred1, y2_test)

0.8341512356687169

In [200]:
mean_squared_error(pred1, y2_test)

2.233821888597599

In [201]:
mean_absolute_error(pred1, y2_test)

0.9487618243900905

In [202]:
mean_euclidean_error(np.column_stack((pred0, pred1)), y_test)

1.4112293294715812

In [203]:
r2_score(np.column_stack((pred0, pred1)), y_test)

0.9045813304591396

In [204]:
mean_squared_error(np.column_stack((pred0, pred1)), y_test)

1.6398189257214744

In [205]:
mean_absolute_error(np.column_stack((pred0, pred1)), y_test)

0.8765812742834569